Make sure you update the **STUDENT_TOKEN**, and then run this block. You should use FIRSTNAME-SURNAME(S) as the TOKEN.

The code in this block can be safely ignored.

# Given code

In [ ]:
#Update your token
STUDENT_TOKEN = '-'

## ignore this code, just used for submission
import requests
import pprint
import json
import random
import math
from copy import copy, deepcopy

def get_problem(problem_id):
  r = requests.get('https://emarchiori.eu.pythonanywhere.com/get-problem?TOKEN=%s&problem=%s' % (STUDENT_TOKEN, problem_id))
  if not r.status_code == 200:
    print('\033[91m' + str(r.status_code))
  return r.json()

def submit_answer(problem_id, answer):
  r = requests.get('https://emarchiori.eu.pythonanywhere.com/submit-answer?TOKEN=%s&problem=%s' % (STUDENT_TOKEN, problem_id), json = answer)
  if not r.status_code == 200:
    print('\033[91m' + str(r.status_code))
  result = r.json()['result']
  if result == 'PASSED':
    print('\033[92m' + result)
  else:
    print('\033[91m' + result)


In [ ]:
problem = get_problem('easy-kropki')

pprint.pprint(problem)

board = problem['board']

print('Input puzzle')
print(board)

# This does not solve the problem, just makes it the same as the input
solution = board

# We print the solution in a way to make it easy to debug and check
print('Solution')
print(solution)

answer = {
    'board': board,
    'id': problem['id']
}

# The answer is submitted to the server, this is just to check the format (it does not check the solution itself)
submit_answer('kropki', answer)

{'board': ['.|.|. | 5|.|. | 8x.|6',
           '- - -   - - x   - - -',
           '.+8|. x 3|7|. + .|1+.',
           '- x +   - + -   - - -',
           '.x4+. | 1|8|6 x .|.|.',
           '+ - -   - - -   + - -',
           '1|.|. | .|.|8 | 2|.|.',
           '- - +   - + -   - + +',
           '8+.|. x .|.|. | .|5|3',
           '- + x   x x -   - - -',
           '5+.|2 x 4+.|. | .|.+8',
           '- + -   - - -   - - +',
           '.|.|8 | .|.|. | .x.|.',
           '+ - +   - + -   - - -',
           '4|1|. | .|.+. x 6|8|5',
           '- x -   + x -   + - -',
           '.|2|. + 8x.+. | 7|.|.'],
 'id': 1}
Input puzzle
['.|.|. | 5|.|. | 8x.|6', '- - -   - - x   - - -', '.+8|. x 3|7|. + .|1+.', '- x +   - + -   - - -', '.x4+. | 1|8|6 x .|.|.', '+ - -   - - -   + - -', '1|.|. | .|.|8 | 2|.|.', '- - +   - + -   - + +', '8+.|. x .|.|. | .|5|3', '- + x   x x -   - - -', '5+.|2 x 4+.|. | .|.+8', '- + -   - - -   - - +', '.|.|8 | .|.|. | .x.|.', '+ - +   - + -   - - -', '4|1|. | .|.+

# PArsing the board and formating for the subission




In [ ]:
import requests
from typing import List, Tuple, Dict


def parse_kropki_board(board: List[str]) -> Tuple[List[List[int]], List[Tuple[Tuple[int, int], Tuple[int, int], str]]]:
    '''Parsing function that outputs the board with locations of numbers and constraints'''
    parsed_board = [[0 for _ in range(9)] for _ in range(9)]
    constraints = []

    row_idx = 0
    for i in range(0, len(board), 2):
        cell_row = board[i]
        constraint_row = board[i + 1] if i + 1 < len(board) else ""

        # Process numbers and dots in cell_rowa
        col_idx = 0  # Column index
        symbol_idx = 0  # Pos in the cell_row string
        while col_idx < 9 and symbol_idx < len(cell_row):
            char = cell_row[symbol_idx]

            if char.isdigit():
                parsed_board[row_idx][col_idx] = int(char)
                col_idx += 1
            elif char == '.':
                parsed_board[row_idx][col_idx] = 0
                col_idx += 1
            elif char in ['+', 'x'] and col_idx > 0:
                # horizontal constraints based on symbol pos
                prev_col = col_idx - 1
                if char == '+':
                    constraints.append(((row_idx, prev_col), (row_idx, col_idx), 'white'))
                elif char == 'x':
                    constraints.append(((row_idx, prev_col), (row_idx, col_idx), 'black'))
            # Ignore `|` or '-'  by not increasing `col_idx` for them

            symbol_idx += 1

        # Process vertical constraints in constraint_row
        if row_idx < 8:  # Ensure not out of rows
            col_idx = 0
            for symbol in constraint_row.split():
                if symbol == '+':
                    constraints.append(((row_idx, col_idx), (row_idx + 1, col_idx), 'white'))
                elif symbol == 'x':
                    constraints.append(((row_idx, col_idx), (row_idx + 1, col_idx), 'black'))
                col_idx += 1

        row_idx += 1

    # Print parsed board for verification
    #print("Parsed 9x9 board with initial values:")
    #for row in parsed_board:
        #print(" | ".join(str(cell) if cell != 0 else "0" for cell in row))

    return parsed_board, constraints


def format_for_submission(parsed_board: List[List[int]], constraints: List[Tuple[Tuple[int, int], Tuple[int, int], str]]) -> List[str]:
    '''Function to convert parsed board and constraints back into the original string format'''
    formatted_board = []
    for row_idx in range(9):
        row_str = ""
        for col_idx in range(9):
            value = parsed_board[row_idx][col_idx]
            row_str += str(value) if value != 0 else "."
            if col_idx < 8:
                # Add a horizontal constraint if present
                next_cell = (row_idx, col_idx + 1)
                constraint = ""
                for pos1, pos2, constraint_type in constraints:
                    if (pos1 == (row_idx, col_idx) and pos2 == next_cell) or (pos2 == (row_idx, col_idx) and pos1 == next_cell):
                        constraint = "+" if constraint_type == "white" else "x"
                        break
                row_str += constraint if constraint else "|"
        formatted_board.append(row_str)

        # Create the constraint row if there are vertical constraints
        if row_idx < 8:
            constraint_row = ""
            for col_idx in range(9):
                above = (row_idx, col_idx)
                below = (row_idx + 1, col_idx)
                constraint = ""
                for pos1, pos2, constraint_type in constraints:
                    if (pos1 == above and pos2 == below) or (pos2 == above and pos1 == below):
                        constraint = "+" if constraint_type == "white" else "x"
                        break
                constraint_row += constraint if constraint else "-"
                if col_idx < 8:
                    constraint_row += " "
            formatted_board.append(constraint_row)

    return formatted_board

# Fetch the problem
problem = get_problem('easy-kropki')
board_data = problem.get('board', [])
pprint.pprint(board_data)
board_id = problem.get('id')

# Parse the board and initialize
parsed_board, parsed_constraints = parse_kropki_board(board_data)

# Convert parsed board and constraints back to the original string format
formatted_board = format_for_submission(parsed_board, parsed_constraints)

print(f"\nFormatted Board for Submission:\nID: {problem['id']}")
for row in formatted_board:
    print(row)


['.|.|. | 5|.|. | 8x.|6',
 '- - -   - - x   - - -',
 '.+8|. x 3|7|. + .|1+.',
 '- x +   - + -   - - -',
 '.x4+. | 1|8|6 x .|.|.',
 '+ - -   - - -   + - -',
 '1|.|. | .|.|8 | 2|.|.',
 '- - +   - + -   - + +',
 '8+.|. x .|.|. | .|5|3',
 '- + x   x x -   - - -',
 '5+.|2 x 4+.|. | .|.+8',
 '- + -   - - -   - - +',
 '.|.|8 | .|.|. | .x.|.',
 '+ - +   - + -   - - -',
 '4|1|. | .|.+. x 6|8|5',
 '- x -   + x -   + - -',
 '.|2|. + 8x.+. | 7|.|.']

Formatted Board for Submission:
ID: 1
.|.|.|5|.|.|8x.|6
- - - - - x - - -
.+8|.x3|7|.+.|1+.
- x + - + - - - -
.x4+.|1|8|6x.|.|.
+ - - - - - + - -
1|.|.|.|.|8|2|.|.
- - + - + - - + +
8+.|.x.|.|.|.|5|3
- + x x x - - - -
5+.|2x4+.|.|.|.+8
- + - - - - - - +
.|.|8|.|.|.|.x.|.
+ - + - + - - - -
4|1|.|.|.+.x6|8|5
- x - + x - + - -
.|2|.+8x.+.|7|.|.


#backtracking



In [ ]:
from typing import List, Tuple
import pprint

def is_valid_assignment(board: List[List[int]], row: int, col: int, num: int, constraints: List[Tuple[Tuple[int, int], Tuple[int, int], str]]) -> bool:
    # Check row and column constraints
    for i in range(9):
        if board[row][i] == num or board[i][col] == num:
            return False

    # Check 3x3 box constraints
    start_row, start_col = 3 * (row // 3), 3 * (col // 3)
    for i in range(3):
        for j in range(3):
            if board[start_row + i][start_col + j] == num:
                return False

    # Check Kropki dot constraints
    for (pos1, pos2, constraint_type) in constraints:
        if (row, col) == pos1 or (row, col) == pos2:
            other_row, other_col = pos2 if (row, col) == pos1 else pos1
            other_val = board[other_row][other_col]

            if other_val != 0:
                if constraint_type == 'white' and abs(num - other_val) != 1:
                    return False
                elif constraint_type == 'black' and (num == 2 * other_val or other_val == 2 * num) == False:
                    return False
    return True

def basic_backtracking_solver(board: List[List[int]], constraints: List[Tuple[Tuple[int, int], Tuple[int, int], str]]) -> Tuple[bool, int]:
    backtrack_count = 0  # Initialize a counter for backtracking steps

    def backtrack() -> bool:
        nonlocal backtrack_count  # Allow access to the outer variable
        # Find the next empty cell (0 represents an empty cell)
        for row in range(9):
            for col in range(9):
                if board[row][col] == 0:
                    # Try each number from 1 to 9
                    for num in range(1, 10):
                        if is_valid_assignment(board, row, col, num, constraints):
                            board[row][col] = num  # Tentative assignment

                            if backtrack():  # Recursively try to complete the puzzle
                                return True
                            board[row][col] = 0  # Undo assignment (backtrack)
                            backtrack_count += 1  # Increment backtrack counter
                    return False  # No valid number found, backtrack
        return True  # Puzzle is complete

    solved = backtrack()
    return solved, backtrack_count


def print_board(board: List[List[int]]) -> None:
    for row in board:
        print(" | ".join(str(cell) if cell != 0 else "0" for cell in row))

# Fetch problem and initialize variables
problem = get_problem('hard-kropki')
board_data = problem.get('board', [])
problem_id = problem.get('id')

print('Input puzzle:')
pprint.pprint(board_data)

# Parse the board into internal representation
parsed_board, parsed_constraints = parse_kropki_board(board_data)

# Solve the puzzle with basic backtracking and get the backtrack count
solved, backtrack_count = basic_backtracking_solver(parsed_board, parsed_constraints)

# Print the results
if solved:
    print(f"Sudoku Kropki puzzle solved with Basic Backtracking:")
    print_board(parsed_board)
else:
    print("No solution found.")

print(f"Total backtracking steps: {backtrack_count}")

# Format the solved board for submission
formatted_solution_board = format_for_submission(parsed_board, parsed_constraints)

# Print the formatted solution for submission
print(f"\nFormatted Board for Submission:\nID: {problem['id']}")
for row in formatted_solution_board:
    print(row)

# Prepare and submit the answer
answer = {'board': formatted_solution_board, 'id': problem_id}
submit_answer('kropki', answer)

Input puzzle:
['.|.x. | .|.|. | .x.|.',
 '- - +   + - -   - + +',
 '.|.|. | .|.x. | .|.|.',
 '- - x   - + -   - - -',
 '.+.|. | 9|.+. | .|.+.',
 '+ - -   + + x   - - -',
 '.|.+. | .x.x. + .|.|.',
 '- - -   - - -   - - -',
 '.x.|. | .|.|. | .|.|.',
 'x - +   - + -   - + x',
 '.+.|. | .|.|. | 5|.|.',
 '- + +   - - -   - + -',
 '.|.|. | .|.|. | .+.|.',
 '- x -   + - -   - - -',
 '.|.|. | .|.+. + .|.+.',
 '- x -   - + +   - x -',
 '.|.|. | .|.|. | .|.|.']
Sudoku Kropki puzzle solved with Basic Backtracking:
1 | 6 | 3 | 7 | 5 | 8 | 2 | 4 | 9
5 | 9 | 4 | 6 | 2 | 1 | 7 | 3 | 8
8 | 7 | 2 | 9 | 3 | 4 | 1 | 5 | 6
7 | 5 | 6 | 8 | 4 | 2 | 3 | 9 | 1
2 | 1 | 9 | 3 | 7 | 5 | 8 | 6 | 4
4 | 3 | 8 | 1 | 6 | 9 | 5 | 7 | 2
6 | 2 | 7 | 4 | 1 | 3 | 9 | 8 | 5
9 | 4 | 1 | 5 | 8 | 7 | 6 | 2 | 3
3 | 8 | 5 | 2 | 9 | 6 | 4 | 1 | 7
Total backtracking steps: 75664

Formatted Board for Submission:
ID: 1
1|6x3|7|5|8|2x4|9
- - + + - - - + +
5|9|4|6|2x1|7|3|8
- - x - + - - - -
8+7|2|9|3+4|1|5+6
+ - - + + x - - -
7|5+6|

# constraint propagation-> forward checking and ac-3


In [ ]:
from typing import List, Tuple, Dict, Set
import pprint
from collections import deque

def initialize_domains(board: List[List[int]]) -> Dict[Tuple[int, int], Set[int]]:
    domains = {}
    for row in range(9):
        for col in range(9):
            if board[row][col] == 0:  # Unassigned cell
                domains[(row, col)] = set(range(1, 10))
            else:
                domains[(row, col)] = {board[row][col]}
    return domains

def forward_check(domains: Dict[Tuple[int, int], Set[int]], row: int, col: int, value: int, constraints: List[Tuple[Tuple[int, int], Tuple[int, int], str]]) -> bool:
    neighbors = [(row, i) for i in range(9) if i != col] + [(i, col) for i in range(9) if i != row]
    box_row, box_col = 3 * (row // 3), 3 * (col // 3)
    neighbors += [(box_row + i, box_col + j) for i in range(3) for j in range(3) if (box_row + i, box_col + j) != (row, col)]

    # Apply Kropki dot constraints to neighbors
    for (pos1, pos2, constraint_type) in constraints:
        if (row, col) == pos1 or (row, col) == pos2:
            other = pos2 if (row, col) == pos1 else pos1
            neighbors.append(other)

    # Propagate by removing the value from the neighbors' domains
    for (r, c) in neighbors:
        if value in domains[(r, c)]:
            domains[(r, c)].discard(value)
            if not domains[(r, c)]:
                return False  # Failure if any domain becomes empty

    return True

def ac3(domains: Dict[Tuple[int, int], Set[int]], constraints: List[Tuple[Tuple[int, int], Tuple[int, int], str]]) -> bool:
    queue = deque([(pos1, pos2) for (pos1, pos2, _) in constraints] + [(pos2, pos1) for (pos1, pos2, _) in constraints])

    while queue:
        (x_i, x_j) = queue.popleft()

        if revise(domains, x_i, x_j, constraints):
            if not domains[x_i]:
                return False  # Domain wiped out; no solution
            # Add all neighbors of x_i back to the queue
            for neighbor in domains:
                if neighbor != x_j and (neighbor, x_i) in queue:
                    queue.append((neighbor, x_i))
    return True

def revise(domains: Dict[Tuple[int, int], Set[int]], x_i: Tuple[int, int], x_j: Tuple[int, int], constraints: List[Tuple[Tuple[int, int], Tuple[int, int], str]]) -> bool:
    revised = False
    for x in domains[x_i].copy():
        if not any(is_valid_pair(x, y, x_i, x_j, constraints) for y in domains[x_j]):
            domains[x_i].remove(x)
            revised = True
    return revised

def is_valid_pair(x: int, y: int, x_i: Tuple[int, int], x_j: Tuple[int, int], constraints: List[Tuple[Tuple[int, int], Tuple[int, int], str]]) -> bool:
    if x == y:  # Basic Sudoku rule for distinct values
        return False
    for (pos1, pos2, constraint_type) in constraints:
        if (x_i, x_j) == (pos1, pos2) or (x_j, x_i) == (pos1, pos2):
            if constraint_type == 'white' and abs(x - y) != 1:
                return False
            elif constraint_type == 'black' and (x == 2 * y or y == 2 * x) == False:
                return False
    return True

def constraint_propagation_solver(board: List[List[int]], constraints: List[Tuple[Tuple[int, int], Tuple[int, int], str]], use_ac3: bool = False) -> Tuple[bool, int]:
    # Initialize domains and optionally run AC-3 to enforce initial arc consistency
    domains = initialize_domains(board)
    if use_ac3:
        if not ac3(domains, constraints):
            return False, 0  # No solution if AC-3 makes a domain empty

    backtrack_count = 0  # Initialize the backtrack counter

    def backtrack(row: int = 0, col: int = 0, domains=domains) -> bool:
        nonlocal backtrack_count  # Allow access to the counter
        # Move to the next unassigned cell
        if row == 9:
            return True  # Puzzle is complete

        next_row, next_col = (row + (col + 1) // 9, (col + 1) % 9)

        # Skip cells that are already filled
        if board[row][col] != 0:
            return backtrack(next_row, next_col, domains)

        # Try each value in the domain of the current cell
        for value in domains[(row, col)].copy():
            if is_valid_assignment(board, row, col, value, constraints):
                board[row][col] = value
                saved_domains = {key: domains[key].copy() for key in domains}

                # Perform forward checking
                domains[(row, col)] = {value}
                if forward_check(domains, row, col, value, constraints) and backtrack(next_row, next_col, domains):
                    return True

                # Backtrack: Restore the board and domains
                board[row][col] = 0
                domains = saved_domains
                backtrack_count += 1  # Increment backtrack count

        return False

    solved = backtrack()
    return solved, backtrack_count


def print_board(board: List[List[int]]) -> None:
    for row in board:
        print(" | ".join(str(cell) if cell != 0 else "0" for cell in row))


# Fetch problem and initialize variables
problem = get_problem('hard-kropki')
board_data = problem.get('board', [])
problem_id = problem.get('id')

print('Input puzzle:')
pprint.pprint(board_data)

# Parse the board into internal representation
parsed_board, parsed_constraints = parse_kropki_board(board_data)

# Solve the puzzle with constraint propagation, forward checking, and optionally AC-3
solved, backtrack_count = constraint_propagation_solver(parsed_board, parsed_constraints, use_ac3=True)

# Print the results
if solved:
    print("Sudoku Kropki puzzle solved with Constraint Propagation, Forward Checking, and AC-3:")
    print_board(parsed_board)
else:
    print("No solution found.")

print(f"\nTotal backtracking steps: {backtrack_count}")

# Format the solved board for submission
formatted_solution_board = format_for_submission(parsed_board, parsed_constraints)

# Print the formatted solution for submission
print(f"\nFormatted Board for Submission:\nID: {problem['id']}")
for row in formatted_solution_board:
    print(row)

# Prepare and submit the answer
answer = {'board': formatted_solution_board, 'id': problem_id}
submit_answer('kropki', answer)

Input puzzle:
['.|.+. | .+.|. | .|.+.',
 '- - x   - - -   - - -',
 '.+.|. x .|.+. | .+.|.',
 '- - x   x - +   - + -',
 '.+.|1 + .|.|. | .x.+.',
 '- x -   - - -   x - x',
 '.|.|. + .|.+. | .|.+.',
 '- + +   + - -   + - +',
 '.x.x. + .|.+. | .|.|.',
 'x - -   - + -   + - -',
 '.|.+. | .+.|. + .|.+.',
 '- - +   - - -   - - x',
 '.|.|. + .|.|. | .|.+.',
 '- x -   x - -   - - -',
 '.|.|. | .+.x2 | .+.|.',
 '- - -   - - -   - - -',
 '.|.|. | .|.+. | .|.|.']
Sudoku Kropki puzzle solved with Constraint Propagation, Forward Checking, and AC-3:
3 | 5 | 4 | 7 | 8 | 6 | 9 | 2 | 1
9 | 8 | 2 | 1 | 3 | 4 | 6 | 5 | 7
7 | 6 | 1 | 2 | 9 | 5 | 8 | 4 | 3
5 | 3 | 9 | 8 | 2 | 1 | 4 | 7 | 6
2 | 4 | 8 | 9 | 6 | 7 | 3 | 1 | 5
1 | 7 | 6 | 4 | 5 | 3 | 2 | 9 | 8
8 | 2 | 7 | 6 | 1 | 9 | 5 | 3 | 4
6 | 1 | 5 | 3 | 4 | 2 | 7 | 8 | 9
4 | 9 | 3 | 5 | 7 | 8 | 1 | 6 | 2

Total backtracking steps: 8399

Formatted Board for Submission:
ID: 0
3|5+4|7+8|6|9|2+1
- - x - - - - - -
9+8|2x1|3+4|6+5|7
- - x x - + - + -
7+6|1+2|9|

# with added MRV

In [ ]:
from typing import List, Tuple, Dict, Set
import pprint
from collections import deque

def initialize_domains(board: List[List[int]]) -> Dict[Tuple[int, int], Set[int]]:
    domains = {}
    for row in range(9):
        for col in range(9):
            if board[row][col] == 0:  # Unassigned cell
                domains[(row, col)] = set(range(1, 10))
            else:
                domains[(row, col)] = {board[row][col]}
    return domains

def forward_check(domains: Dict[Tuple[int, int], Set[int]], row: int, col: int, value: int, constraints: List[Tuple[Tuple[int, int], Tuple[int, int], str]]) -> bool:
    neighbors = [(row, i) for i in range(9) if i != col] + [(i, col) for i in range(9) if i != row]
    box_row, box_col = 3 * (row // 3), 3 * (col // 3)
    neighbors += [(box_row + i, box_col + j) for i in range(3) for j in range(3) if (box_row + i, box_col + j) != (row, col)]

    # Apply Kropki dot constraints to neighbors
    for (pos1, pos2, constraint_type) in constraints:
        if (row, col) == pos1 or (row, col) == pos2:
            other = pos2 if (row, col) == pos1 else pos1
            neighbors.append(other)

    # Propagate by removing the value from the neighbors' domains
    for (r, c) in neighbors:
        if value in domains[(r, c)]:
            domains[(r, c)].discard(value)
            if not domains[(r, c)]:
                return False  # Failure if any domain becomes empty

    return True

def ac3(domains: Dict[Tuple[int, int], Set[int]], constraints: List[Tuple[Tuple[int, int], Tuple[int, int], str]]) -> bool:
    queue = deque([(pos1, pos2) for (pos1, pos2, _) in constraints] + [(pos2, pos1) for (pos1, pos2, _) in constraints])

    while queue:
        (x_i, x_j) = queue.popleft()

        if revise(domains, x_i, x_j, constraints):
            if not domains[x_i]:
                return False  # Domain wiped out; no solution
            # Add all neighbors of x_i back to the queue
            for neighbor in domains:
                if neighbor != x_j and (neighbor, x_i) in queue:
                    queue.append((neighbor, x_i))
    return True

def revise(domains: Dict[Tuple[int, int], Set[int]], x_i: Tuple[int, int], x_j: Tuple[int, int], constraints: List[Tuple[Tuple[int, int], Tuple[int, int], str]]) -> bool:
    revised = False
    for x in domains[x_i].copy():
        if not any(is_valid_pair(x, y, x_i, x_j, constraints) for y in domains[x_j]):
            domains[x_i].remove(x)
            revised = True
    return revised

def is_valid_pair(x: int, y: int, x_i: Tuple[int, int], x_j: Tuple[int, int], constraints: List[Tuple[Tuple[int, int], Tuple[int, int], str]]) -> bool:
    if x == y:  # Basic Sudoku rule for distinct values
        return False
    for (pos1, pos2, constraint_type) in constraints:
        if (x_i, x_j) == (pos1, pos2) or (x_j, x_i) == (pos1, pos2):
            if constraint_type == 'white' and abs(x - y) != 1:
                return False
            elif constraint_type == 'black' and (x == 2 * y or y == 2 * x) == False:
                return False
    return True

def select_unassigned_variable_mrv(domains: Dict[Tuple[int, int], Set[int]], board: List[List[int]]) -> Tuple[int, int]:
    min_domain_size = float('inf')
    best_cell = None
    for cell, domain in domains.items():
        row, col = cell
        if board[row][col] == 0:  # Only consider unassigned cells
            domain_size = len(domain)
            if domain_size < min_domain_size:
                min_domain_size = domain_size
                best_cell = cell
    return best_cell

def constraint_propagation_solver(board: List[List[int]], constraints: List[Tuple[Tuple[int, int], Tuple[int, int], str]], use_ac3: bool = False) -> Tuple[bool, int]:
    # Initialize domains and optionally run AC-3 to enforce initial arc consistency
    domains = initialize_domains(board)
    if use_ac3:
        if not ac3(domains, constraints):
            return False, 0  # No solution if AC-3 makes a domain empty

    backtrack_count = 0  # Initialize the backtrack counter

    def backtrack() -> bool:
        nonlocal backtrack_count, domains  # Allow access to domains and counter
        # Select the next cell using MRV
        cell = select_unassigned_variable_mrv(domains, board)
        if cell is None:
            return True  # Puzzle is complete

        row, col = cell

        # Try each value in the domain of the current cell
        for value in domains[(row, col)].copy():
            if is_valid_assignment(board, row, col, value, constraints):
                board[row][col] = value
                saved_domains = {key: domains[key].copy() for key in domains}

                # Perform forward checking
                domains[(row, col)] = {value}
                if forward_check(domains, row, col, value, constraints) and backtrack():
                    return True

                # Backtrack: Restore the board and domains
                board[row][col] = 0
                domains = saved_domains
                backtrack_count += 1  # Increment backtrack count

        return False

    solved = backtrack()
    return solved, backtrack_count

def print_board(board: List[List[int]]) -> None:
    for row in board:
        print(" | ".join(str(cell) if cell != 0 else "0" for cell in row))


# Fetch problem and initialize variables
problem = get_problem('hard-kropki')
board_data = problem.get('board', [])
problem_id = problem.get('id')

print('Input puzzle:')
pprint.pprint(board_data)

# Parse the board into internal representation
parsed_board, parsed_constraints = parse_kropki_board(board_data)

# Solve the puzzle with constraint propagation, forward checking, MRV, and optionally AC-3
solved, backtrack_count = constraint_propagation_solver(parsed_board, parsed_constraints, use_ac3=True)

# Print the results
if solved:
    print("Sudoku Kropki puzzle solved with Constraint Propagation, Forward Checking, MRV, and AC-3:")
    print_board(parsed_board)
else:
    print("No solution found.")

print(f"\nTotal backtracking steps: {backtrack_count}")

# Format the solved board for submission
formatted_solution_board = format_for_submission(parsed_board, parsed_constraints)

# Print the formatted solution for submission
print(f"\nFormatted Board for Submission:\nID: {problem['id']}")
for row in formatted_solution_board:
    print(row)

# Prepare and submit the answer
answer = {'board': formatted_solution_board, 'id': problem_id}
submit_answer('kropki', answer)

Input puzzle:
['.|.x. | .|.|. | .x.|.',
 '- - +   + - -   - + +',
 '.|.|. | .|.x. | .|.|.',
 '- - x   - + -   - - -',
 '.+.|. | 9|.+. | .|.+.',
 '+ - -   + + x   - - -',
 '.|.+. | .x.x. + .|.|.',
 '- - -   - - -   - - -',
 '.x.|. | .|.|. | .|.|.',
 'x - +   - + -   - + x',
 '.+.|. | .|.|. | 5|.|.',
 '- + +   - - -   - + -',
 '.|.|. | .|.|. | .+.|.',
 '- x -   + - -   - - -',
 '.|.|. | .|.+. + .|.+.',
 '- x -   - + +   - x -',
 '.|.|. | .|.|. | .|.|.']
Sudoku Kropki puzzle solved with Constraint Propagation, Forward Checking, MRV, and AC-3:
1 | 6 | 3 | 7 | 5 | 8 | 2 | 4 | 9
5 | 9 | 4 | 6 | 2 | 1 | 7 | 3 | 8
8 | 7 | 2 | 9 | 3 | 4 | 1 | 5 | 6
7 | 5 | 6 | 8 | 4 | 2 | 3 | 9 | 1
2 | 1 | 9 | 3 | 7 | 5 | 8 | 6 | 4
4 | 3 | 8 | 1 | 6 | 9 | 5 | 7 | 2
6 | 2 | 7 | 4 | 1 | 3 | 9 | 8 | 5
9 | 4 | 1 | 5 | 8 | 7 | 6 | 2 | 3
3 | 8 | 5 | 2 | 9 | 6 | 4 | 1 | 7

Total backtracking steps: 14997

Formatted Board for Submission:
ID: 1
1|6x3|7|5|8|2x4|9
- - + + - - - + +
5|9|4|6|2x1|7|3|8
- - x - + - - - -
8+7|

# normal sudoku


In [ ]:
from typing import List, Tuple, Dict, Set
import pprint
from collections import deque

def initialize_domains(board: List[List[int]]) -> Dict[Tuple[int, int], Set[int]]:
    domains = {}
    for row in range(9):
        for col in range(9):
            if board[row][col] == 0:  # Unassigned cell
                domains[(row, col)] = set(range(1, 10))
            else:
                domains[(row, col)] = {board[row][col]}
    return domains

def forward_check(domains: Dict[Tuple[int, int], Set[int]], row: int, col: int, value: int, constraints: List[Tuple[Tuple[int, int], Tuple[int, int], str]]) -> bool:
    neighbors = [(row, i) for i in range(9) if i != col] + [(i, col) for i in range(9) if i != row]
    box_row, box_col = 3 * (row // 3), 3 * (col // 3)
    neighbors += [(box_row + i, box_col + j) for i in range(3) for j in range(3) if (box_row + i, box_col + j) != (row, col)]

    # Propagate by removing the value from the neighbors' domains
    for (r, c) in neighbors:
        if value in domains[(r, c)]:
            domains[(r, c)].discard(value)
            if not domains[(r, c)]:
                return False  # Failure if any domain becomes empty

    return True

def ac3(domains: Dict[Tuple[int, int], Set[int]], constraints: List[Tuple[Tuple[int, int], Tuple[int, int], str]]) -> bool:
    queue = deque([(pos1, pos2) for (pos1, pos2, _) in constraints] + [(pos2, pos1) for (pos1, pos2, _) in constraints])

    while queue:
        (x_i, x_j) = queue.popleft()

        if revise(domains, x_i, x_j, constraints):
            if not domains[x_i]:
                return False  # Domain wiped out; no solution
            # Add all neighbors of x_i back to the queue
            for neighbor in domains:
                if neighbor != x_j and (neighbor, x_i) in queue:
                    queue.append((neighbor, x_i))
    return True

def revise(domains: Dict[Tuple[int, int], Set[int]], x_i: Tuple[int, int], x_j: Tuple[int, int], constraints: List[Tuple[Tuple[int, int], Tuple[int, int], str]]) -> bool:
    revised = False
    for x in domains[x_i].copy():
        if not any(is_valid_pair(x, y, x_i, x_j, constraints) for y in domains[x_j]):
            domains[x_i].remove(x)
            revised = True
    return revised

def is_valid_pair(x: int, y: int, x_i: Tuple[int, int], x_j: Tuple[int, int], constraints: List[Tuple[Tuple[int, int], Tuple[int, int], str]]) -> bool:
    return x != y  # For regular Sudoku, only check that values are distinct

def select_unassigned_variable_mrv(domains: Dict[Tuple[int, int], Set[int]], board: List[List[int]]) -> Tuple[int, int]:
    min_domain_size = float('inf')
    best_cell = None
    for cell, domain in domains.items():
        row, col = cell
        if board[row][col] == 0:  # Only consider unassigned cells
            domain_size = len(domain)
            if domain_size < min_domain_size:
                min_domain_size = domain_size
                best_cell = cell
    return best_cell

def constraint_propagation_solver(board: List[List[int]], constraints: List[Tuple[Tuple[int, int], Tuple[int, int], str]], use_ac3: bool = False) -> Tuple[bool, int]:
    # Initialize domains and optionally run AC-3 to enforce initial arc consistency
    domains = initialize_domains(board)
    if use_ac3:
        if not ac3(domains, constraints):
            return False, 0  # No solution if AC-3 makes a domain empty

    backtrack_count = 0  # Initialize the backtrack counter

    def backtrack() -> bool:
        nonlocal backtrack_count, domains  # Allow access to domains and counter
        # Select the next cell using MRV
        cell = select_unassigned_variable_mrv(domains, board)
        if cell is None:
            return True  # Puzzle is complete

        row, col = cell

        # Try each value in the domain of the current cell
        for value in domains[(row, col)].copy():
            if is_valid_assignment(board, row, col, value, []):
                board[row][col] = value
                saved_domains = {key: domains[key].copy() for key in domains}

                # Perform forward checking
                domains[(row, col)] = {value}
                if forward_check(domains, row, col, value, []) and backtrack():
                    return True

                # Backtrack: Restore the board and domains
                board[row][col] = 0
                domains = saved_domains
                backtrack_count += 1  # Increment backtrack count

        return False

    solved = backtrack()
    return solved, backtrack_count

# Sample standard Sudoku puzzle
sudoku_puzzle = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]

print("Original Sudoku Puzzle:")
print_board(sudoku_puzzle)

# Solve the puzzle with constraint propagation, forward checking, MRV, and optionally AC-3
solved, backtrack_count = constraint_propagation_solver(sudoku_puzzle, [], use_ac3=True)

# Print the results
if solved:
    print("Sudoku puzzle solved:")
    print_board(sudoku_puzzle)
else:
    print("No solution found.")

print(f"\nTotal backtracking steps: {backtrack_count}")

Original Sudoku Puzzle:
5 3 . . 7 . . . .
6 . . 1 9 5 . . .
. 9 8 . . . . 6 .
8 . . . 6 . . . 3
4 . . 8 . 3 . . 1
7 . . . 2 . . . 6
. 6 . . . . 2 8 .
. . . 4 1 9 . . 5
. . . . 8 . . 7 9
Sudoku puzzle solved:
5 3 4 6 7 8 9 1 2
6 7 2 1 9 5 3 4 8
1 9 8 3 4 2 5 6 7
8 5 9 7 6 1 4 2 3
4 2 6 8 5 3 7 9 1
7 1 3 9 2 4 8 5 6
9 6 1 5 3 7 2 8 4
2 8 7 4 1 9 6 3 5
3 4 5 2 8 6 1 7 9

Total backtracking steps: 1754


# without recursion but with stack

In [ ]:
from typing import List, Tuple

def initialize_domains(board: List[List[int]]) -> Dict[Tuple[int, int], Set[int]]:
    domains = {}
    for row in range(9):
        for col in range(9):
            if board[row][col] == 0:  # Unassigned cell
                domains[(row, col)] = set(range(1, 10))
            else:
                domains[(row, col)] = {board[row][col]}
    return domains

def select_unassigned_variable_mrv(board: List[List[int]], domains: Dict[Tuple[int, int], Set[int]]) -> Tuple[int, int]:
    min_domain_size = float('inf')
    best_cell = None
    for cell, domain in domains.items():
        row, col = cell
        if board[row][col] == 0:  # Only consider unassigned cells
            domain_size = len(domain)
            if domain_size < min_domain_size:
                min_domain_size = domain_size
                best_cell = cell
    return best_cell

def backtracking_search_with_mrv_non_recursive(board: List[List[int]], constraints: List[Tuple[Tuple[int, int], Tuple[int, int], str]]) -> Tuple[bool, int]:
    backtrack_count = 0  # Counter for backtracking steps
    stack = []  # Stack to keep track of state (row, col, attempted values)

    # Initialize domains for MRV
    domains = initialize_domains(board)

    # Start by pushing the first empty cell to the stack with no attempted values
    cell = select_unassigned_variable_mrv(board, domains)
    if cell is None:
        return True, backtrack_count  # Puzzle is already solved

    row, col = cell
    stack.append((row, col, []))  # (row, col, list of attempted values)

    while stack:
        # Current position and the values already tried for this cell
        row, col, tried_values = stack.pop()

        # Find the next valid number for this cell
        found_valid = False
        for num in range(1, 10):
            if num not in tried_values and is_valid_assignment(board, row, col, num, constraints):
                board[row][col] = num
                stack.append((row, col, tried_values + [num]))  # Update attempted values
                found_valid = True
                break

        # If found a valid number, move to the next cell with MRV
        if found_valid:
            next_cell = select_unassigned_variable_mrv(board, domains)
            if next_cell is None:
                return True, backtrack_count  # Puzzle is complete
            next_row, next_col = next_cell
            stack.append((next_row, next_col, []))
        else:
            # Backtrack if no valid number found, clear the cell
            board[row][col] = 0
            backtrack_count += 1

    return False, backtrack_count  # No solution found

def print_board(board: List[List[int]]) -> None:
    for row in board:
        print(" | ".join(str(cell) if cell != 0 else "0" for cell in row))


# Example usage
problem = get_problem('hard-kropki')
board_data = problem.get('board', [])
problem_id = problem.get('id')

print('Input puzzle:')
pprint.pprint(board_data)

# Parse the board into internal representation
parsed_board, parsed_constraints = parse_kropki_board(board_data)

# Solve the puzzle using non-recursive backtracking with MRV
solved, backtrack_count = backtracking_search_with_mrv_non_recursive(parsed_board, parsed_constraints)

# Print the results
if solved:
    print("Sudoku Kropki puzzle solved with non-recursive:")
    print_board(parsed_board)
else:
    print("No solution found.")

print(f"Total backtracking steps (non-recursive): {backtrack_count}")

# Format the solved board for submission
formatted_solution_board = format_for_submission(parsed_board, parsed_constraints)

# Prepare the answer for submission
print(f"\nFormatted Board for Submission:\nID: {problem['id']}")
pprint.pprint(formatted_solution_board)
answer = {
    'board': formatted_solution_board,
    'id': problem_id
}

# Submit the answer
submit_answer('kropki', answer)

Input puzzle:
['.|.x. | .|.|. | .x.|.',
 '- - +   + - -   - + +',
 '.|.|. | .|.x. | .|.|.',
 '- - x   - + -   - - -',
 '.+.|. | 9|.+. | .|.+.',
 '+ - -   + + x   - - -',
 '.|.+. | .x.x. + .|.|.',
 '- - -   - - -   - - -',
 '.x.|. | .|.|. | .|.|.',
 'x - +   - + -   - + x',
 '.+.|. | .|.|. | 5|.|.',
 '- + +   - - -   - + -',
 '.|.|. | .|.|. | .+.|.',
 '- x -   + - -   - - -',
 '.|.|. | .|.+. + .|.+.',
 '- x -   - + +   - x -',
 '.|.|. | .|.|. | .|.|.']
Sudoku Kropki puzzle solved with non-recursive:
1 | 6 | 3 | 7 | 5 | 8 | 2 | 4 | 9
5 | 9 | 4 | 6 | 2 | 1 | 7 | 3 | 8
8 | 7 | 2 | 9 | 3 | 4 | 1 | 5 | 6
7 | 5 | 6 | 8 | 4 | 2 | 3 | 9 | 1
2 | 1 | 9 | 3 | 7 | 5 | 8 | 6 | 4
4 | 3 | 8 | 1 | 6 | 9 | 5 | 7 | 2
6 | 2 | 7 | 4 | 1 | 3 | 9 | 8 | 5
9 | 4 | 1 | 5 | 8 | 7 | 6 | 2 | 3
3 | 8 | 5 | 2 | 9 | 6 | 4 | 1 | 7
Total backtracking steps (non-recursive): 75664

Formatted Board for Submission:
ID: 1
['1|6x3|7|5|8|2x4|9',
 '- - + + - - - + +',
 '5|9|4|6|2x1|7|3|8',
 '- - x - + - - - -',
 '8+7|2|9|3+4|1